In [12]:
import pandas as pd
from glob import glob
import os
import numpy as np
from tqdm import tqdm, tqdm_notebook


import torch
from torch import nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import gluonnlp as nlp
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model
from transformers import AdamW

ModuleNotFoundError: No module named 'mxnet'

In [3]:
DATA_PATH = f"D:\code\DACON-Korean-sentence-relation-classification\open\open"
train = pd.read_csv(os.path.join(DATA_PATH, "train_data.csv"))
test = pd.read_csv(os.path.join(DATA_PATH, "test_data.csv") )
submission = pd.read_csv(os.path.join(DATA_PATH, "sample_submission.csv"))

In [7]:
max_len = 70
batch_size = 128
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate = 5e-5

device = torch.device("cuda:0")